<a href="https://colab.research.google.com/github/satyamsingh1004/ML/blob/master/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**


*   In case of basic cells ( a single recurrent nueron or a layer of recurrent nuerons ),  the output is simply equal to the state , but in more complex cells this is not always the case. A cells hidden state and output may be different

*  [Rnn Types](https://drive.google.com/open?id=1xnpAwujeR1ymXfqgg7HGy9KQnPETHZhN)
        Sequence to sequence: To predict Time series, such as stock prices
        Sequence to vector: to output sentiment score
        Vector to sequence: to generate image caption
        Encoder(sequence to vector) & Decoder(vector to sequence:) translating a sentence from one language to another


*   **tf.Variable** vs  **tf.Placeholder**
    tf.Variable is used for trainable variables such as weights and biases of model
    tf.Placeholder is used to feed actual training examples
*   Only after running **tf.global_variables_initializer()** in a session will your variables hold the values you told them to hold when you declare them (tf.Variable(tf.zeros(...)), tf.Variable(tf.random_normal(...)),...).


        


   



In [0]:
import tensorflow as tf

import numpy as np

In [0]:
n_inputs = 3
n_nuerons = 5


In [0]:
X0 = tf.placeholder(tf.float32,[None,n_inputs])
X1 = tf.placeholder(tf.float32,[None,n_inputs])

In [0]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_nuerons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_nuerons,n_nuerons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_nuerons],dtype=tf.float32))

In [0]:
Y0 = tf.tanh(tf.matmul(X0,Wx) + b)
Y1 = tf.tanh(tf.matmul(X1,Wx) + tf.matmul(Y0,Wy)  + b)

In [0]:
#init = tf.global_variables_initializer()

In [0]:
# Mini-batch: instance 0,instance 1,instance 2,instance 3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

In [0]:
#with tf.Session() as sess:
  #init.run()
  #Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [0]:
#print(Y0_val)

**Static unrolling through time**

When you are using
basic cells, the final state is simply equal to the last output.

In [0]:
#tf.reset_default_graph()

#X0 = tf.placeholder(tf.float32,[None,n_inputs])
#X1 = tf.placeholder(tf.float32,[None,n_inputs])

#basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_nuerons)
#output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32)

#Y0,Y1 = output_seqs
#S0,S1 = states

#print('Outputs: ', Y0)
#print('States: ', S0)

In [0]:
tf.reset_default_graph()
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_nuerons)
output_seqs, states = tf.contrib.rnn.static_rnn(
basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])
## work on states later

W0702 10:08:24.580158 140477367760768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0702 10:08:24.581925 140477367760768 deprecation.py:323] From <ipython-input-11-989a8ae219c7>:6: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0702 10:08:24.583628 140477367760768 deprecation.py:323] From <ipython-input-11-989a8ae219c7>:8: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Pl

In [0]:
X_batch = np.array([
# t = 0 t = 1
[[0, 1, 2], [9, 8, 7]], # instance 0
[[3, 4, 5], [0, 0, 0]], # instance 1
[[6, 7, 8], [6, 5, 4]], # instance 2
[[9, 0, 1], [3, 2, 1]], # instance 3
])

In [0]:
#init = tf.global_variables_initializer()
#with tf.Session() as sess:
  #init.run()
  #outputs_val = outputs.eval(feed_dict={X: X_batch})

In [0]:
#print(outputs_val)

Dynamic Unrolling Through Time

In [0]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_nuerons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
sequence_length=seq_length)

W0702 10:08:24.806543 140477367760768 deprecation.py:323] From <ipython-input-15-09180b75c696>:7: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0702 10:08:24.912296 140477367760768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
X_batch = np.array([
# step 0 step 1
[[0, 1, 2], [9, 8, 7]], # instance 0
[[3, 4, 5], [0, 0, 0]], # instance 1 (padded with a zero vector)
[[6, 7, 8], [6, 5, 4]], # instance 2
[[9, 0, 1], [3, 2, 1]], # instance 3
])
seq_length_batch = np.array([2, 1, 2, 2])

In [0]:
#init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  outputs_val, states_val = sess.run(
  [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

NameError: ignored

In [0]:
print(outputs_val)

MNIST using RNNs

In [0]:
from tensorflow.contrib.layers import fully_connected

In [0]:
tf.reset_default_graph()

n_steps = 28
n_inputs = 28

n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X= tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None])

with tf.variable_scope('rnn', initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)):
  basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) 
  outputs,states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

  logits = fully_connected(states,n_outputs,activation_fn = None)
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
  loss = tf.reduce_mean(xentropy)
  optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)

  training_op = optimizer.minimize(loss)
  correct = tf.nn.in_top_k(logits,y,1)
  accuracy = tf.reduce_mean((tf.cast(correct,tf.float32)))

In [0]:
init = tf.global_variables_initializer()

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets("tmp/data/")

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
X_test = mnist.test.images.reshape((-1,n_steps,n_inputs))
y_test = mnist.test.labels

In [0]:
n_epochs = 100
batch_size = 150

In [0]:
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      X_batch = X_batch.reshape((-1,n_steps,n_inputs))
      sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:X_test,y:y_test})
    print(epoch,"Train accuracy: ",acc_train,"; Test accuracy: ", acc_test)
      
    

Time Series Prediction


*   an OutputProjectionWrapper is the simplest solution to reduce the
dimensionality of the RNN’s output sequences down to just one value per time step
(per instance), it is not the most efficient.
*   List item







In [0]:
tf.reset_default_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

#cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

n_iterations = 10000
batch_size = 50

with tf.Session() as sess:
  init.run()
  for iteration in range(n_iterations):
    X_batch, y_batch = 



In [0]:
n_steps = 10

sequence = [0.] * n_steps
for iteration in range(300):
  X_batch = np.array(sequence[-n_steps:]).reshape(1, n_steps, 1)
  y_pred = sess.run(outputs, feed_dict={X: X_batch})
  sequence.append(y_pred[0, -1, 0])

RuntimeError: ignored